<a href="https://colab.research.google.com/github/ProfEngel/OpenTuneWeaver/blob/main/OpenTuneWeaver_lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning eines LLM

Mithilfe dieses Colab-Notebooks kann man mit wenigen Schritten ein kleines Sprachmodell auf eigene Anweisungen feinabstimmen. Dafür sind ein paar Vorarbeiten wichtig. Daher bitte Schritt für Schritt dieses Notebook durchgehen und wenn erforderlich Anpassungen vornehmen.

Diese Anleitung funktioniert vollständig in der freien colab-Version. Bitte beachten: Für die Laufzeit muss eine GPU gewählt werden. In der freien Version wäre das die T4 NVIDIA GPU. Diese auswählen und starten....

Viel Erfolg!

# 1. Vorbereiten des Q&A-Datensatzes
1.   Mit T4-GPU (mindestens) eine Laufzeit starten.
2.   Nun das Dokument für das Finetuning wie folgt hochladen: **data.pdf**

## 1.1 Einrichtung der Arbeitsumgebung und Start des Ollama-Servers
Dieser Codeblock ist dafür zuständig, die Arbeitsumgebung für die Verwendung der Ollama KI-API vorzubereiten und den Ollama-Server zu starten. Hier eine kurze Erklärung der Schritte:

* Installation notwendiger Pakete: Zuerst werden einige Linux-Pakete mit apt-get install und das Python-Paket PyPDF2 installiert. PyPDF2 ist eine Bibliothek zur Arbeit mit PDF-Dateien in Python. Das Installieren dieser Pakete ermöglicht die Nutzung von spezifischer Software und Bibliotheken, die für die folgenden Schritte benötigt werden.

* Herunterladen und Einrichten von Ollama: Mit dem Befehl curl wird ein Skript von der Ollama-Webseite heruntergeladen und ausgeführt, das für die Installation und Einrichtung der Ollama KI-API sorgt. Dies ermöglicht die Verwendung der Ollama-KI-Funktionalitäten direkt im Notebook.

* Starten des Ollama-Servers: Durch die Definition und Ausführung der ollama-Funktion in einem separaten Thread wird der Ollama-Server gestartet. Dieser Server ermöglicht es, Anfragen an die Ollama KI-API zu senden und Antworten zu erhalten. Die Umgebungsvariablen OLLAMA_HOST und OLLAMA_ORIGINS werden gesetzt, um den Server zu konfigurieren.

* Herunterladen von KI-Modellen: Der Befehl ollama pull mistral lädt das KI-Modell „mistral“ herunter, das für die Verarbeitung der Anfragen verwendet wird. Die auskommentierten Zeilen zeigen, wie man weitere Modelle herunterladen könnte, diese sind jedoch in diesem Beispiel nicht aktiviert.

Insgesamt bereitet dieser Codeblock alles Nötige vor, um den Ollama-Server zu starten und Anfragen an die Ollama KI-API zu senden. Dies ermöglicht es, fortgeschrittene KI-Funktionalitäten wie Textanalyse oder Frage-Antwort-Systeme direkt im Notebook zu nutzen.

In [3]:
%%capture
! sudo apt-get install -y pciutils
!pip install pypdf2
! curl https://ollama.ai/install.sh | sh
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

! ollama pull llama3.1
#! ollama pull mixtral
#! ollama pull llama2:13b

#ollama_thread = threading.Thread(target=ollama)
#ollama_thread.start()

## 1.2 Test der KI-Anfrage mit Ollama
Dieser Codeblock führt einen Test durch, um zu überprüfen, ob die Einrichtung erfolgreich war und der Ollama-Server richtig funktioniert. Er sendet eine spezifische Frage an die KI und zeigt die Antwort an. Hier sind die Schlüsselschritte erklärt:


*   Definieren der Anfrage: Es wird ein Text (prompt) definiert, der die Frage enthält, welche die KI beantworten soll. In diesem Fall geht es um eine kurze Erklärung zum Klimawandel. Dieser Text wird an die KI gesendet, um zu sehen, wie sie komplexe Fragen in einer einfachen und verständlichen Form beantwortet.

*  Vorbereitung der Anfrage: Die Anfrage wird als payload vorbereitet, das die Details zur Anfrage, wie das zu verwendende KI-Modell (mixtral), die Temperatur für die Antwortgenerierung (0.6), und die Nachrichten enthält, die den Rahmen für die KI-Antwort setzen. Hier wird die KI als "hilfreicher KI-Assistent" angesprochen, um den Kontext für die Antwort zu geben.

*  Senden der Anfrage: Die Anfrage wird mithilfe der requests.post-Funktion an die Ollama-API gesendet. Die API-Adresse (url) verweist auf den lokal gestarteten Ollama-Server.

*  Anzeigen der Antwort: Nach dem Empfang der Antwort vom Server wird diese von einem Byte-String in ein JSON-Objekt umgewandelt (json.loads). Die Antwort der KI auf die gestellte Frage wird dann auf der Konsole ausgegeben.

Dieser Test ist ein wichtiger Schritt, um sicherzustellen, dass alles korrekt eingerichtet ist und die KI wie erwartet auf Anfragen reagiert. Es hilft dabei, eventuelle Probleme in der Einrichtung oder Kommunikation mit dem Ollama-Server frühzeitig zu erkennen und zu beheben.

In [4]:
prompt = """
Was bedeutet der Klimawandel?
Kannst Du das in drei Sätzen erklären!
"""

url = 'http://localhost:11434/api/chat'
payload = {
    "model": "llama3.1",
    "temperature": 0.6,
    "stream": False,
    "messages": [
        {"role": "system", "content": "Du bist ein hilfreicher KI-Assistent!"},
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(url, json=payload)
message_str = response.content.decode('utf-8')
message_dict = json.loads(message_str)
print(message_dict['message']['content'])

Der Klimawandel bezeichnet die Zunahme der durchschnittlichen Temperaturen auf der Erde über einen längeren Zeitraum, verursacht hauptsächlich durch den Anstieg von Kohlendioxid (CO2) und anderen Treibhausgasen in der Atmosphäre. Dieser Prozess führt zu einer globale Erwärmung, die weitreichige Auswirkungen auf das Klima und Ökosysteme hat, einschließlich steigender Meeresspiegel, vermehrter extreme Wetterereignisse und veränderten Niederschlagsmuster.

Der Klimawandel ist durch eine Reihe von Faktoren verursacht, darunter die Verbrennung fossiler Brennstoffe wie Kohle, Öl und Gas, die Abholzung von Wäldern sowie die Landwirtschaft. Die Folgen des Klimawandels sind vielfältig und reichen von der Zerstörung von Lebensräumen bis hin zu den Auswirkungen auf die menschliche Gesundheit und Wirtschaft.


## 1.3 Vorbereitung und Analyse eines PDF-Dokuments für Feinabstimmung
Dieser Codeblock leitet einen Prozess zur Vorbereitung eines PDF-Dokuments für die Feinabstimmung mit einer KI ein, indem er Fragen und Antworten basierend auf dem Inhalt des Dokuments generiert. Der Prozess ist in mehrere Schlüsselschritte unterteilt:

*   Text aus PDF extrahieren: Zunächst wird der Text aus der hochgeladenen PDF-Datei (data.pdf) extrahiert. Benutzer sollten darauf achten, eine PDF mit dem Namen data.pdf hochzuladen. Der Text wird dann für die weitere Verarbeitung gespeichert.

*   Text in Abschnitte unterteilen: Um die Bearbeitung zu vereinfachen und die Belastung für die KI zu reduzieren, wird der Text in kleinere Abschnitte unterteilt. Diese Aufteilung hilft, den Text besser handhabbar zu machen und optimiert den Prozess der Frage-Antwort-Erzeugung.

*   Frage-Antwort-Paare erzeugen: Für jeden Textabschnitt werden spezifische Frage-Antwort-Paare generiert. Dies geschieht durch das Senden des Textes an die Ollama KI-API, die instruiert wird, präzise und relevante Fragen und Antworten basierend auf dem Textinhalt zu formulieren.

*   Ausgabe der aktuellen Q&A: Während des Prozesses wird jede generierte Frage-Antwort-Paarung ausgegeben. Dies ermöglicht eine sofortige Überprüfung und gewährleistet, dass die Ergebnisse wie erwartet sind. Es bietet auch die Möglichkeit, den Fortschritt zu verfolgen.

*   Speichern der Ergebnisse: Am Ende werden alle generierten Frage-Antwort-Paare in einer Datei (data_qa.json) gespeichert. Diese Datei kann heruntergeladen, überarbeitet und für die Feinabstimmung der KI oder andere Zwecke verwendet werden.

Dieser Prozess ist besonders nützlich für Anwender, die ihre KI-Modelle mit spezifischem Wissen aus Dokumenten anreichern oder trainieren möchten. Die generierten Q&A-Paare dienen als Basis für das Feintuning, wodurch die KI in der Lage ist, auf ähnliche Fragen in Zukunft genauer und kontextbezogener zu antworten.

In [5]:
import PyPDF2
import json
import requests

def extract_text_from_pdf(file_path):
    pdf_file_obj = open(file_path, 'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
    text = ''
    for page_num in range(len(pdf_reader.pages)):
        page_obj = pdf_reader.pages[page_num]
        text += page_obj.extract_text()
    pdf_file_obj.close()
    return text

def split_text_into_chunks(text, chunk_size=512):
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

def submit_to_ollama_api(prompt, retries=3):
    url = 'http://localhost:11434/api/chat'
    detailed_instruction = (
        "Basierend auf dem folgenden Text, formuliere bitte klare und präzise Frage-Antwort-Paare. "
        "Die Fragen sollten mit 'Im Kontext ...' beginnen, um den Bezug zum Text deutlich zu machen. "
        "Antworten sollten direkt aus dem Text abgeleitet und wortwörtlich übernommen werden. "
        "Ziel ist es, die Informationen im Text durch diese Q&A zugänglich zu machen. "
        "Hier ist ein Beispiel, wie deine Antworten aussehen sollten:\n\n"
        "Hier zunächst der Beispieltext aus dem Q&A erzeugt werden:\n"
        "Erneuerbare Energien wie Solar- und Windkraft werden immer wichtiger, um den Klimawandel zu bekämpfen. Im Gegensatz zu fossilen Brennstoffen, die begrenzt sind, bieten erneuerbare Energien eine nachhaltige Alternative, die die Umwelt schont. Zudem sind die Kosten für die Erzeugung von Strom aus erneuerbaren Quellen in den letzten Jahren stark gesunken, was zu einer höheren Akzeptanz und Verbreitung geführt hat.\n"
        "Hier ein paar Beispiele für Frage-Antwort-Paare:\n"
        "Frage: Im Kontext der erneuerbaren Energien, warum werden Solar- und Windkraft immer wichtiger?\n"
        "Antwort: Erneuerbare Energien wie Solar- und Windkraft werden immer wichtiger, um den Klimawandel zu bekämpfen.\n"
        "Frage: Im Kontext der Energiequellen, was ist der Vorteil von erneuerbaren Energien im Vergleich zu fossilen Brennstoffen?\n"
        "Antwort: Der Vorteil von erneuerbaren Energien ist, dass sie eine nachhaltige Alternative bieten, im Gegensatz zu fossilen Brennstoffen, die begrenzt sind. Während fossile Brennstoffe bei der Verbrennung CO₂ freisetzen, das für den Klimawandel verantwortlich ist, sind erneuerbare Energien emissionsfrei. Darüber hinaus schont ihre Nutzung die Umwelt, und sie sind unerschöpflich verfügbar, was ihre Bedeutung langfristig noch erhöht.\n"
        "Frage: Im Kontext der erneuerbaren Energien, was hat zur höheren Akzeptanz von erneuerbaren Energien geführt?\n"
        "Antwort: Die Kosten für die Erzeugung von Strom aus erneuerbaren Quellen sind in den letzten Jahren stark gesunken. Dieser Preisrückgang hat erneuerbare Energien wettbewerbsfähiger gemacht und ihre Akzeptanz sowohl bei Konsumenten als auch bei Unternehmen erhöht. Zusätzlich haben staatliche Förderprogramme und Subventionen dazu beigetragen, dass erneuerbare Energien eine zentrale Rolle in der Energieversorgung einnehmen. Die Kombination aus Kostensenkungen und politischen Maßnahmen hat entscheidend zur Verbreitung von erneuerbaren Energien beigetragen.\n"
    )
    payload = {
        "model": "llama3.1",
        "temperature": 0.5,
        "stream": False,
        "messages": [
            {"role": "system", "content": detailed_instruction},
            {"role": "user", "content": prompt}
        ]
    }
    for i in range(retries):
        try:
            response = requests.post(url, json=payload)
            if response.status_code == 200:
                message_str = response.content.decode('utf-8')
                message_dict = json.loads(message_str)
                if 'content' in message_dict.get('message', {}):
                    print("Aktuelle Q&A:")
                    print(message_dict['message']['content'])
                    print("-" * 80)  # Eine Trennlinie für bessere Lesbarkeit
                    return message_dict['message']['content']
                else:
                    print("No valid content found in the response.")
            else:
                print(f"Attempt {i + 1} failed with status code {response.status_code}. Retrying...")
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
    print("Max retries exceeded. Skipping this prompt.")
    return None

text = extract_text_from_pdf('data.pdf')
chunks = split_text_into_chunks(text, 512)

responses = []
for chunk in chunks:
    response = submit_to_ollama_api(chunk)
    if response:
        responses.append(response)
        print("Aktuelle Q&A:")
        print(response)
        print("-" * 80)  # Eine Trennlinie für bessere Lesbarkeit

# Optional: Speichern der Antworten in einer Datei oder weitere Verarbeitung
with open('data_qa.json', 'w', encoding='utf-8') as f:
    json.dump(responses, f, ensure_ascii=False, indent=4)


SyntaxError: unterminated string literal (detected at line 28) (<ipython-input-5-b24be78c740f>, line 28)



---


# WICHTIG!!!! Zwischenschritt

1.   Q&A-Datensatz herunterladen (data_qa.json)
2.   Datensatz überarbeiten (fehlerhafte Elemente löschen/anpassen, bereinigen)
3.   Notebook zurücksetzen
4.   Datensatz hochladen
5.   Notebook neu starten. Wieder mit T4-GPU



---



# 2. Finetuning

Das folgende Skript geht zurück auf [Unsloth](https://github.com/unslothai/unsloth). Einem aktuellen Framework um auch mit wenigen Ressourcen schnell qualitativ hochwertige Ergebnisse zu erzielen.

Hier der Link zum Original-Notebook von [Mistral7b-Unsloth](https://colab.research.google.com/drive/1Dyauq4kTZoLewQ1cApceUQVNcnnNTzg_?usp=sharing)

## 2.1 Installation nötiger Bibs und Bezug des Modells



*  Zunächst müssen relevante Python-Bibs bezogen werden.
*  Das Modell Mistral-7b wird heruntergeladen und in 4bit bereitgestellt.
*  Das Lora-Adapter-Template wird heruntergeladen um für das Finetuning trainiert zu werden.
*  Der Datensatz wird geladen und in das passende Format übertragen

In [ ]:
%%capture
!pip install datasets
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
import json

# Laden des lokalen Datensatzes
with open('data_qa.json', 'r', encoding='utf-8') as file:
    data_qa = json.load(file)

EOS_TOKEN = "[EOS]"  # Beispiel für ein EOS-Token, anpassen nach Bedarf

def formatting_prompts_func(data_qa):
    texts = []
    for qa_pair in data_qa:
        # Direkte Verwendung des QA-Paares als "instruction" und "output", "input" bleibt leer
        instruction = "Bitte umformulieren der folgenden Frage und Antwort in ein klares Format."
        input = ""  # Kein spezifischer Eingabetext, da der QA-Text bereits die Information enthält
        output = qa_pair  # Direkte Verwendung des QA-Paares aus dem Datensatz
        text = f"### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n{output}" + EOS_TOKEN
        texts.append(text)
    return texts

# Verarbeitung und Formatierung des Datensatzes
formatted_data = formatting_prompts_func(data_qa)

# Beispielweise Ausgabe eines formatierten Elements zur Überprüfung
print(formatted_data[0])


In [ ]:
from datasets import Dataset

# Angenommen, formatted_data ist eine Liste von formatierten Texten
# Beispiel für formatted_data:
# formatted_data = ["### Instruction:\nFrage ... Antwort ... [EOS]", "..."]

# Konvertieren von formatted_data in ein Dataset-Objekt
dataset = Dataset.from_dict({"text": formatted_data})


## 2.2 Training

(das Training kann 10-30 Minuten dauern, je nach Größe des Datensatzes auch länger)

*  Hyperparameter für das Training (können angepasst werden um das Training zu optimieren)
*  Darstellen der Ressourcen vor und nach dem Training
*  Trainingsverlauf (hier den angegebenen Wert beachten. Dies ist der Verlustwert und er sollte gegen 0 gehen. Sollte dieser steigen, so wäre es nachteilig für das Training)

In [ ]:
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

# Konvertieren von formatted_data in ein Dataset-Objekt (vorausgesetzt, formatted_data ist eine Liste von Strings)
dataset = Dataset.from_dict({"text": formatted_data})

# Anschließende Anpassungen und Initialisierung des Trainers
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,  # Verwenden Sie das konvertierte Dataset-Objekt hier
    dataset_text_field="text",
    max_seq_length=128,  # Stellen Sie sicher, dass diese Zahl geeignet ist für Ihre Daten
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
    ),
)


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats (optional)
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

## 2.3 Speichern des Modells und des Adapters

(dies kann bis 20 Minuten dauern)

*  Zunächst wird der trainierte Adapter gespeichert
*  Nun wird das Modell im 4b-quantisierten Format *.gguf gespeichert

In [ ]:
model.save_pretrained("lora_model") # Local saving
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# Save to q4_k_m GGUF
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")



---


# WICHTIG!!!! Zwischenschritt

1.   model-unsloth.Q4_K_M.gguf herunterladen
2.   Notebook mit T4-GPU neu starten
3.   model-unsloth.Q4_K_M.gguf hochladen
4.   Ab hier weitermachen!



---



# 3. Inferenz (Test)

## 3.1 Testvorbereitungen



*  Instalation aller relevanten Python-Bibs für Ollama
*  Erstellen eines Ollama-Modells basierend auf dem trainierten *.gguf-Modells

In [ ]:
!rm model-unsloth.F16.gguf
!rm -r model
!rm -r lora_model
!rm -r llama.cpp
!rm -r outputs

In [ ]:
%%capture
! sudo apt-get install -y pciutils
!pip install pypdf2
! curl https://ollama.ai/install.sh | sh
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
%%capture
# Dateiname definieren
filename = 'Modelfile'

# Inhalt der Datei
content = 'FROM ./model-unsloth.Q4_K_M.gguf'

# Datei erstellen und schreiben
with open(filename, 'w') as file:
    file.write(content)

print(f"Datei '{filename}' wurde erfolgreich erstellt.")

!ollama create ft_mistral -f Modelfile

import subprocess

# Befehl definieren
command = 'ollama create ft_mistral -f Modelfile'

# Befehl ausführen
process = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Ausgabe und Fehlerausgabe (falls vorhanden) anzeigen
print(process.stdout.decode())
if process.stderr:
    print("Fehler:", process.stderr.decode())


## 3.2 Testdurchführung mit eigenen Prompts



*  Eigenen Prompt eingeben um das Modell zu testen

In [ ]:
# Benutzeraufforderung für den gesamten Prompt
prompt = input("Bitte geben Sie Ihren Prompt ein: ")

# Ausgabe des Prompts zur Überprüfung
print("Ihr eingegebener Prompt lautet:")
print(prompt)

url = 'http://localhost:11434/api/chat'
payload = {
    "model": "ft_mistral",
    "temperature": 0.6,
    "stream": False,
    "messages": [
        {"role": "system", "content": "Du bist ein hilfreicher KI-Assistent!"},
        {"role": "user", "content": prompt}
    ]
}

response = requests.post(url, json=payload)
message_str = response.content.decode('utf-8')
message_dict = json.loads(message_str)
print(message_dict['message']['content'])